In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import re
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
import os
data_path=r"C:\Users\7ashk\Downloads";
file1 = os.path.join(data_path, 'words.txt')
file2 = os.path.join(data_path, 'words_250000_train.txt')
output_file = os.path.join(data_path, 'combined_unique.txt')


for f in (file1, file2):
    if not os.path.exists(f):
        raise FileNotFoundError(f"Couldn't find {f}")

unique_words = set()

for fname in (file1, file2):
    with open(fname, 'r', encoding='utf-8') as f:
        for line in f:
            word = line.strip()
            if not word or ' ' in word:
                continue
            unique_words.add(word)
            if '-' in word:
                parts = word.split('-')
                for part in parts:
                    if part and ' ' not in part:
                        unique_words.add(part)
                concat = ''.join(parts)
                if concat and ' ' not in concat:
                    unique_words.add(concat)
            else:
                unique_words.add(word)
out_dir = os.path.dirname(output_file)
if out_dir and not os.path.isdir(out_dir):
    os.makedirs(out_dir)

with open(output_file, 'w', encoding='utf-8') as out:
    for w in unique_words:
        out.write(w + '\n')

print(f"Written {len(unique_words)} unique words to '{output_file}'")


Written 548064 unique words to 'C:\Users\7ashk\Downloads\combined_unique.txt'


In [ ]:
_ALPHA_RE="abcdefghijklmnopqrstuvwxyz"

In [ ]:
def expand_hyphenated(token: str, *, min_len: int = 2, max_len: int = 25) -> List[str]:
    
    if "-" not in token:
        return []

    pieces = token.split("-")
    out: set[str] = set()

    # individual pieces
    for p in pieces:
        p = p.lower()
        if min_len <= len(p) <= max_len and _ALPHA_RE.fullmatch(p):
            out.add(p)

    # concatenated form
    concat = "".join(pieces).lower()
    if min_len <= len(concat) <= max_len and _ALPHA_RE.fullmatch(concat):
        out.add(concat)

    return list(out)

In [ ]:
def load_clean_words(
    path: str | Path,
    *,
    min_len: int = 2,
    max_len: int = 25,
    sample: int = 20,
) -> Tuple[list[str], list[str]]:
    path = Path(path)

    clean: list[str] = []
    rejected: list[str] = []

    with path.open("r", encoding="utf-8", errors="ignore") as fh:
        for raw in fh:
            raw = raw.strip()
            if not raw:
                continue

            raw_low = raw.lower()

            if "-" in raw_low:
                # Expand and record original as rejected
                expanded = expand_hyphenated(raw_low, min_len=min_len, max_len=max_len)
                clean.extend(expanded)
                rejected.append(raw_low)
                continue

            if min_len <= len(raw_low) <= max_len and _ALPHA_RE.fullmatch(raw_low):
                clean.append(raw_low)
            else:
                rejected.append(raw_low)

    # Deduplicate while preserving order
    clean_unique = list(dict.fromkeys(clean))
    rejected_unique = list(dict.fromkeys(rejected))

    print(
        f"Loaded {len(clean_unique):,} clean words (" f"{len(rejected_unique):,} rejected)"
    )
    if sample and rejected_unique:
        print("Sample rejected:", rejected_unique[:sample])

    return clean_unique, rejected_unique


In [32]:
from typing import Iterable

In [ ]:
def words_to_dataframe(words: Iterable[str]) -> "pd.DataFrame":
    df = pd.DataFrame({"word": list(words)})
    df["len"] = df.word.str.len()
    return df

In [35]:
txt_path = output_file

In [36]:
words, rejected = load_clean_words(txt_path)

Loaded 456,198 clean words (50,664 rejected)
Sample rejected: ['spoon-fed', 'dim-brooding', 'gospel-true', 'tub-t', "coyote's", 'half-grecized', 'almain-rivets', 'stout-bodied', 'co.', 'uncross-examined', "haunch's", 'pilot-boat', 'slow-breathing', 'try-', 'snaky-tailed', 'ill-friended', 'grim-looking', 'self-collectedness', 'franco-provencal', 'second-guess']


In [ ]:
def save_words_to_file(words: Iterable[str], path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        for w in words:
            f.write(w + "\n")

In [50]:
words, _ = load_clean_words(txt_path)
save_words_to_file(words, r"C:\Users\7ashk\Downloads\final_words.txt")

Loaded 456,198 clean words (50,664 rejected)
Sample rejected: ['spoon-fed', 'dim-brooding', 'gospel-true', 'tub-t', "coyote's", 'half-grecized', 'almain-rivets', 'stout-bodied', 'co.', 'uncross-examined', "haunch's", 'pilot-boat', 'slow-breathing', 'try-', 'snaky-tailed', 'ill-friended', 'grim-looking', 'self-collectedness', 'franco-provencal', 'second-guess']
